<a href="https://colab.research.google.com/github/kolomichyk/BMSTU-Hack/blob/develop/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Инициализация проекта

In [ ]:
#!g1.4
import catboost as cb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import Normalizer
import pickle

### Ввод данных и обработка данных

In [ ]:
#!g1.4
data = pd.read_csv('data_train.csv')
target = pd.read_csv('train_target.csv')

In [ ]:
#!g1.4
data['target'] = target
data = data.dropna()
data = data.drop(columns=['distance', 'vidsobst'])

wagnum             0
prev_fr_id         0
prev_is_load       0
prev_snd_org_id    0
prev_rsv_org_id    0
prev_distance      0
snd_st_id          0
rsv_st_id          0
fr_id              0
is_load            0
common_ch          0
snd_org_id         0
rsv_org_id         0
prev_freight       0
prev_fr_group      0
freight            0
fr_group           0
Travel_time        0
Mounth             0
dotw               0
time_depart        0
target             0
dtype: int64

In [ ]:
#!g1.4
target = data['target']
data = data.drop(columns=['target'])

(1560446, 21) (1560446,)


In [ ]:
#!g1.4
data = data.drop(columns=['time_depart'])

### Train Test Split

In [ ]:
#!g1.4
X_train, X_test, y_train, y_test = train_test_split(
    data,
    target,
    test_size=0.1,
    random_state=42
)

### Создание и обучение моделей на основе catboost

In [ ]:
#!g1.4
cat_features = ['prev_freight', 'prev_fr_group', 'freight', 'fr_group']
one_hot_max_size = 10 
intervals = [5, 10]
model1 = cb.CatBoostRegressor(
    cat_features=cat_features,
    iterations=5000,
    depth=10,
    learning_rate=1,
    random_seed=128,
    use_best_model=True,
    loss_function = "MAE"
)
    

In [ ]:
#!g1.4
model1.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
#!g1.4
model2 = cb.CatBoostRegressor(
    cat_features=cat_features,
    iterations=3000,
    depth=8,
    learning_rate=1.0,
    random_seed=42,
    use_best_model=True,
    loss_function = "MAE"
)

In [ ]:
#!g1.4
model2.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
#!g1.4
model3 = cb.CatBoostRegressor(
    cat_features=cat_features,
    iterations=1000,
    depth=5,
    learning_rate=1,
    random_seed=42,
    use_best_model=True,
    loss_function = "MAE"
)

In [ ]:
#!g1.4
model3.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
#!g1.4
model4 = cb.CatBoostRegressor(
    cat_features=cat_features,
    iterations=2000,
    depth=8,
    learning_rate=1,
    random_seed=128,
    use_best_model=True,
    loss_function = "MAE"
)

In [ ]:
#!g1.4
model4.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
#!g1.4
def FindAccuracyBagging(y_test, y_pred1, y_pred2, y_pred3, y_pred4):
    return (np.abs(y_test-(y_pred1+y_pred2+y_pred3+y_pred4)/4) < 12).sum() / len(y_test)

In [ ]:
#!g1.4
FindAccuracyBagging(y_test, model4.predict(X_test), model3.predict(X_test), model2.predict(X_test), model.predict(X_test))

In [ ]:
#!g1.1
model5 = cb.CatBoostRegressor(
    cat_features=cat_features,
    iterations=500,
    depth=16,
    learning_rate=0.5,
    random_seed=60,
    use_best_model=True,
    loss_function = "MAE"
)

In [ ]:
#!g1.1
model5.fit(X_train, y_train, eval_set=(X_test, y_test))

In [ ]:
#!g1.4
def FindAccuracy(y_test, y_pred):
    return (np.abs(y_test - y_pred) < 12).sum() / len(y_test)

In [ ]:
#!g1.4
FindAccuracy(y_test, model5.predict(X_test))

### Выгрузка лучшей модели

In [ ]:
#!g1.4
with open('model1.pickle', 'wb') as f:
    pickle.dump(model5, f)

### Выгрузка итоговых данных

In [ ]:
#!g1.4
pd.DataFrame(model.predict(X_test)).to_csv('submit.csv', sep=',', index=False)